In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [ ]:
!export hf_token="hf_..."

In [ ]:
SAGEMAKER_IAM_ROLE_NAME = ""

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName=SAGEMAKER_IAM_ROLE_NAME)['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'mistralai/Mixtral-8x7B-Instruct-v0.1',
    'SM_NUM_GPUS': json.dumps(8)  # Change as per requirements
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface",version="1.4.2"),
    env=hub,
    role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.48xlarge",
    container_startup_health_check_timeout=300,
)
  
# send request
predictor.predict({
    "inputs": "My name is Clara and I am",
})